## [분야1] 완전 자율 네트워크 운영을 위한 초단기 네트워크 트래픽 예측 베이스라인 코드
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.

### 추론 실행 환경
1. python 3.10 환경
2. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    - 최신 버전입니다.

코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 필요 라이브러리를 직접 설치합니다.
2. 추론용 코드 작성
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. aif.submit() 함수를 호출하여 최종 결과를 제출하는 파트입니다.
    - 마이 페이지에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
    - 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [7]:
!pip install -U aifactory

In [9]:
# 자신의 모델에 필요한 추가 라이브러리 설치
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0
!pip install joblib
!pip install numpy
!pip install pandas
!pip install neuralforecast
!pip install scipy

### 2. 추론 환경의 기본 경로 구조
#### 제출 시 주의사항

1. 테스트 데이터 경로: /aif/data/test_inputs.pkl
   - 채점에 사용될 테스트 입력 데이터는 이 디렉토리 안에 test_inputs.pkl이라는 이름으로 고정되어 제공됩니다.
2. 모델 및 자원 경로: 예시 : ./model/
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)
3. 제출 파일은 submission.pkl로 저장
4. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
5. return 할 결과물과 양식에 유의합니다.

In [ ]:
import joblib           # pip install joblib
import numpy as np                                  # pip install numpy
import pandas as pd                                 # pip install pandas
from neuralforecast.core import NeuralForecast      # pip install neuralforecast

def load_parameter():
    nf = NeuralForecast.load(path='model')
    return nf

nf = load_parameter()
input_data = joblib.load("/aif/data/test_inputs.pkl")
max_i = input_data.shape[0]
n = 100
batch_size = 50
future_max_list = []

for start_idx in range(0, max_i, batch_size):
    end_idx = min(start_idx + batch_size, max_i)
    batch_indices = range(start_idx, end_idx)
    
    # batch용 DataFrame 생성
    batch_dfs = []
    for i in batch_indices:
        test_df = pd.DataFrame({
            'unique_id': [f'test_series_{i}'] * n,
            'ds': pd.date_range('2025-01-01', periods=n, freq='100ms'),
            'y': input_data[i, :, 6]  # i번째 샘플의 y (shape=(100,))
        })
        batch_dfs.append(test_df)
    
    # batch의 모든 시계열을 하나로 concat
    batch_df = pd.concat(batch_dfs, ignore_index=True)
    
    # batch 단위 예측 수행
    forecasts = nf.predict(df=batch_df)
    
    # unique_id별로 max 계산 후 future_max_list에 저장
    max_per_series = forecasts.groupby('unique_id')['NHITS'].max()
    # max_per_series의 인덱스가 'test_series_i' 형태이므로 i를 다시 얻어서 리스트에 저장
    for unique_id, max_val in max_per_series.items():
        # unique_id는 'test_series_i'꼴이므로 i 추출
        idx = int(unique_id.split('_')[-1])
        # 리스트에 위치 맞게 저장 (append 대신에 확실하게 인덱스 맞춰서 저장 가능)
        if len(future_max_list) <= idx:
            future_max_list.extend([None]*(idx - len(future_max_list) + 1))
        future_max_list[idx] = max_val

# submission.pkl 파일로 저장
future_max_array = np.array(future_max_list).reshape(-1, 1)  # shape: (163854,)
joblib.dump(future_max_array, "submission.pkl")

c:\Users\ksy01\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-02 22:26:36,918	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-08-02 22:26:37,136	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Seed set to 42


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\CNU_Program\\network\\2025_Network_AI_hackathon\\Task1\\aif\\data\\test_inputs.pkl'

### 3. 제출하기 
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 대회 페이지 [베이스라인 코드](https://aifactory.space/task/9162/baseline) 탭에 기재된 가이드라인을 따라 task별로 확인하실 수 있습니다.
- key가 틀리면 제출이 진행되지 않거나 잘못 제출되므로 태스크에 맞는 자기 key를 사용해야 합니다.

In [11]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="Project_Test",
           key="fa9d96d6-4d7d-4d42-bee8-0874c24f5030"
           )
#-----------------------------------------------------#
print(time.time() - t)

file : task.py
python
제출 완료
13.132704973220825
